In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Input
from tensorflow.keras.models import Model

In [2]:
# 1. 예제 입력 시퀀스 준비
# 시퀀스: [1, 2, 3,4]
# shape: (batch_size=1, timesteps=4, features=1)
x = np.array([[1, 2, 3, 4]], dtype=np.float32)
x = x.reshape(1, 4, 1)

In [3]:
# 2. LSTM 모델 정의
# hidden size를 2로 둬서 상태 벡터를 눈으로 보기 좋게 만든다.
inputs = Input(shape=(4,1))
lstm = LSTM(
    units=2,
    return_sequences=True, # 모든 time step의 출력 반환
    return_state=True      # 마지막 hidden state, cell state도 반환
)

# outputs_seq: (batch, timesteps, units)
# h_last: 마지막 hidden state (batch, units)
# c_last: 마지막 cell state (batch, units)
outputs_seq, h_last, c_last = lstm(inputs)

model = Model(inputs=inputs, outputs=[outputs_seq, h_last, c_last])

In [4]:
# 3. 한 번 forward 해보기
outputs_seq_val, h_last_val, c_last_val = model.predict(x)

print("입력 시퀀스:")
print(x[0, :, 0])  # [1. 2. 3. 4.]

print("\n각 time step의 출력(hidden state):")
# outputs_seq_val.shape = (1, 4, 2)
for t in range(outputs_seq_val.shape[1]):
  print(f"t={t+1}, h_t = {outputs_seq_val[0, t]}")

print("\n마지막 hidden state h_T:")
print(h_last_val[0])

print("\n마지막 cell state c_T:")
print(c_last_val[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
입력 시퀀스:
[1. 2. 3. 4.]

각 time step의 출력(hidden state):
t=1, h_t = [0.06984076 0.05178567]
t=2, h_t = [0.1872146  0.08337323]
t=3, h_t = [0.28572685 0.07912552]
t=4, h_t = [0.3246414  0.05521701]

마지막 hidden state h_T:
[0.3246414  0.05521701]

마지막 cell state c_T:
[0.3672137  0.23879863]


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [6]:
#1. 데이터 만들기
#   -각 시퀀스 길이 3
#   -각 원소는 0~9 사이의 정스
#   -라벨은 세 수의 합

num_samples = 5000 #샘플 수
seq_len = 3 #시퀀스 길이
max_val = 10 #숫자 범위: 0~9

#(num_samples, seqlen)
X_int = np.random.randint(0, max_val, size=(num_samples, seq_len))

# 라벨: 세 수의 합 -> (num_samples, 1)
y = np.sum(X_int, axis=1, keepdims=True).astype("float32")

#LSTM 입력 형태로 reshape: (batch, timesteps, features)
#여기서는 feature가 "한 숫자"이므로 1
X = X_int.reshape(num_samples, seq_len, 1).astype("float32")

#train / test split
split = int(num_samples * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print("X_train shape:", X_train.shape) # (4000, 3 1)
print("y_train shape:", y_train.shape) # (4000, 1)

X_train shape: (4000, 3, 1)
y_train shape: (4000, 1)


In [7]:
# 2. LSTM 모델 정의
model = Sequential([
    LSTM(16, input_shape=(seq_len, 1)), #3 step 시퀀스, feature 1개
    Dense(1)  #출력: 합 (스칼라)
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 16)             │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,169 (4.57 KB)

 Trainable params: 1,169 (4.57 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# 3 학습
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 214.3038 - mae: 13.6827 - val_loss: 167.7975 - val_mae: 12.0099
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 145.9966 - mae: 11.0199 - val_loss: 98.6217 - val_mae: 8.8454
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 86.3054 - mae: 8.0832 - val_loss: 53.2160 - val_mae: 6.1000
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 46.9415 - mae: 5.6055 - val_loss: 35.5068 - val_mae: 4.8344
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 32.6291 - mae: 4.5703 - val_loss: 26.3783 - val_mae: 4.1023
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 25.2110 - mae: 3.9528 - val_loss: 20.0664 - val_mae: 3.4905
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 18.7451 - mae: 3.3556 - val_loss: 15.2966 - val_mae: 2.9404
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14.3468 - mae: 2.7847 - val_loss: 11.3158 - val_mae: 2.3809
Epoch 9/20
100/100 ━━━━━━━━━━━━━━━

In [10]:
# 성능 확인
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest MSE: {loss: .4f}, Test MAE: {mae: .4f}")


Test MSE:  1.0850, Test MAE:  0.5713


In [12]:
# 5. 몇 개 예시로 예측 확인
def demo_example(seq):
  """
  seq: 길이 3인 정수 리스트 예) [3, 5, 2]
  """
  arr = np.array(seq, dtype="float32").reshape(1, seq_len, 1)
  pred = model.predict(arr, verbose=0)[0, 0]
  true_sum = sum(seq)
  print(f"입력 시퀀스: {seq}")
  print(f"정답(세 수의 합): {true_sum}")
  print(f"LSTM 예측값 : {pred: .2f}")
  print("-" * 40)

demo_example([1, 2, 3])
demo_example([4, 0, 9])
demo_example([7, 7, 7])

입력 시퀀스: [1, 2, 3]
정답(세 수의 합): 6
LSTM 예측값 :  5.94
----------------------------------------
입력 시퀀스: [4, 0, 9]
정답(세 수의 합): 13
LSTM 예측값 :  12.96
----------------------------------------
입력 시퀀스: [7, 7, 7]
정답(세 수의 합): 21
LSTM 예측값 :  19.49
----------------------------------------
